In [55]:
from dataclasses import dataclass
import numpy as np
from matplotlib import pyplot as plt
import plotly.graph_objects as go

NDArray = np.ndarray

In [56]:
# Order of detection result
# 0,   1,     2,    3
# TL,  TR,    BR,   BL
# RED, GREEN, BLUE, YELLOW


@dataclass
class DiamondBoardParameter:
    marker_leghth: float
    """
    the ArUco marker length in meter
    """
    chess_length: float
    """
    the length of the chess board in meter
    """
    border_length: float = 0.01
    """
    border_length in m, default is 1cm
    """

    @property
    def marker_border_length(self):
        assert self.chess_length > self.marker_leghth
        return (self.chess_length - self.marker_leghth) / 2

    @property
    def total_side_length(self):
        assert self.chess_length > self.marker_leghth
        return self.marker_border_length * 2 + self.chess_length * 3


# 9mm + 127mm + 127mm (97mm marker) + 127mm + 10mm
# i.e. marker boarder = 127mm - 97mm = 30mm (15mm each side)
Point2D = tuple[float, float]
Quad2D = tuple[Point2D, Point2D, Point2D, Point2D]


class ArUcoMarker2D:
    id: int
    _corners: NDArray

    def __init__(self, id: int, corners: Quad2D):
        self.id = id
        tmp = np.array(corners, dtype=np.float32)
        assert tmp.shape == (4, 2)
        self._corners = tmp

    @property
    def corners(self):
        return self._corners
    
    @property
    def center(self):
        return np.mean(self.corners, axis=0)


class ArUcoMarker3D:
    id: int
    _corners: NDArray

    def __init__(self, id: int, corners: NDArray):
        self.id = id
        tmp = np.array(corners, dtype=np.float32)
        assert tmp.shape == (4, 3)
        self._corners = tmp

    @staticmethod
    def from_2d(marker2d: ArUcoMarker2D, z: float = 0.0):
        return ArUcoMarker3D(
            marker2d.id, np.column_stack((marker2d.corners, np.full((4,), z)))
        )

    @property
    def corners(self):
        return self._corners

    @property
    def center(self):
        return np.mean(self.corners, axis=0)

    def normal(self, length: float = 1):
        """
        return (2, 3)
        """
        x, y, _ = self.center
        return np.array([(x, y, 0), (x, y, length)])


# let's let TL be the origin
def generate_diamond_corners(
    ids: tuple[int, int, int, int], params: DiamondBoardParameter
):
    """
    A diamond chess board, which could be count as a kind of ChArUco board

    C | 0 | C
    ---------
    1 | C | 2
    ---------
    C | 3 | C

    where C is the chess box, and 0, 1, 2, 3 are the markers (whose ids are passed in order)

    Args:
        ids: a tuple of 4 ids of the markers
        params: DiamondBoardParameter
    """

    def tl_to_square(tl_x: float, tl_y: float, side_length: float) -> Quad2D:
        return (
            (tl_x, tl_y),
            (tl_x + side_length, tl_y),
            (tl_x + side_length, tl_y + side_length),
            (tl_x, tl_y + side_length),
        )

    tl_0_x = params.border_length + params.chess_length + params.marker_border_length
    tl_0_y = params.border_length + params.marker_border_length

    tl_1_x = params.border_length + params.marker_border_length
    tl_1_y = params.border_length + params.chess_length + params.marker_border_length

    tl_2_x = (
        params.border_length + params.chess_length * 2 + params.marker_border_length
    )
    tl_2_y = tl_1_y

    tl_3_x = params.border_length + params.chess_length + params.marker_border_length
    tl_3_y = (
        params.border_length + params.chess_length * 2 + params.marker_border_length
    )
    return (
        ArUcoMarker2D(
            ids[0],
            tl_to_square(tl_0_x, tl_0_y, params.marker_leghth),
        ),
        ArUcoMarker2D(
            ids[1],
            tl_to_square(tl_1_x, tl_1_y, params.marker_leghth),
        ),
        ArUcoMarker2D(
            ids[2],
            tl_to_square(tl_2_x, tl_2_y, params.marker_leghth),
        ),
        ArUcoMarker2D(
            ids[3],
            tl_to_square(tl_3_x, tl_3_y, params.marker_leghth),
        ),
    )

In [57]:
params = DiamondBoardParameter(0.097, 0.127)
markers = generate_diamond_corners((16, 17, 18, 19), params)

In [60]:
fig = go.Figure()
for marker in markers:
    corners = marker.corners
    center = marker.center
    corners = np.append(corners, [center], axis=0)
    fig.add_trace(go.Scatter(x=corners[:, 0], y=corners[:, 1], mode='lines+markers', name=f"Marker {marker.id}"))

# set the aspect ratio as 1:1
fig.update_layout(
    width=600,
    height=600,
)
fig.update_yaxes(autorange="reversed")
fig.show()

In [61]:
markers_3d = [ArUcoMarker3D.from_2d(m) for m in markers]
fig = go.Figure()
for m in markers_3d:
    fig.add_trace(
        go.Scatter3d(
            x=m.corners[:, 0],
            y=m.corners[:, 1],
            z=m.corners[:, 2],
            mode="lines+markers+text",
            marker=dict(size=2),
            line=dict(width=2),
            textposition="top center",
            text=m.id,
            name=f"Marker {m.id}",
        )
    )
    n = m.normal(0.1)
    fig.add_trace(
        go.Scatter3d(
            x=n[:, 0],
            y=n[:, 1],
            z=n[:, 2],
            mode="lines",
            line=dict(width=2),
            name=f"Normal {m.id}",
        )
    )
# note that the Y axis is reversed
fig.update_layout(scene=dict(yaxis_autorange="reversed"))
fig.show()